In [9]:
#!/usr/bin/env python
# coding: utf-8

#Import all the stuff
import csv
import sys
import pandas as pd
import numpy as np
import sklearn
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
import math
import datetime
from nltk.tokenize import TweetTokenizer

#Import file
file = r'.\all\train.csv'
d = pd.read_csv(file)
test_file = r'.\all\test.csv'
test_d = pd.read_csv(test_file)

data = d[["text", "created", "label", "retweetCount"]] #Full dataset
test_data = test_d[["text", "created", "retweetCount"]]

# print (data)
stopWrds = stopwords.words('english')

def formatTraining(x, whichFeat):
    spl = x.split("/")
    if len(spl[0]) == 1:
        spl[0] = "0" + spl[0]
    if len(spl[1]) == 1:
        spl[1] = "0" + spl[1]
    x_new = "/".join(spl)
    if whichFeat == "weekday":
        return datetime.datetime.strptime(x_new, '%m/%d/%y %H:%M').weekday()
    elif whichFeat == "hour":
        return datetime.datetime.strptime(x_new, '%m/%d/%y %H:%M').hour
     

def formatTest(x, whichFeat):
    spl = x.split("/")
    if len(spl[0]) == 1:
        spl[0] = "0" + spl[0]
    if len(spl[1]) == 1:
        spl[1] = "0" + spl[1]
    x_new = "/".join(spl)
    if whichFeat == "weekday":
        return datetime.datetime.strptime(x_new, '%m/%d/%Y %H:%M').weekday()
    elif whichFeat == "hour":
        return datetime.datetime.strptime(x_new, '%m/%d/%Y %H:%M').hour

def formatTimes(x, time):
    return 1 if time == x else 0

def removeStops(x):
    spl = x.split()
    spl = [item for item in spl if item.lower() not in stopWrds]
    return ' '.join(spl)

def removeLinks(x):
    if "https" in x:
        x = x.split("https")
        return x[0] + "https"
    return x

fmtTime = np.vectorize(formatTimes)
rmvStops = np.vectorize(removeStops)
rmvLinks = np.vectorize(removeLinks)
    
vals = data["created"].values
fmtTrain = np.vectorize(formatTraining)
day_created_single = fmtTrain(vals, "weekday")
hour_created_single = fmtTrain(vals, "hour")
data.loc[:, "text"] = data["text"].apply(rmvStops)
data.loc[:, "text"] = data["text"].apply(rmvLinks)

#create binary matrix for weekdays for training
n = day_created_single.shape[0]

day_created = np.zeros((n, 7))
for i in range(7):
    day_created[:,i] = fmtTime(day_created_single, i)

hour_created = np.zeros((n, 24))
for i in range(24):
    hour_created[:, i] = fmtTime(hour_created_single, i)

# test_data.loc[:, "text"] = test_data["text"].apply(getHTTP)
fmtTest = np.vectorize(formatTest)
day_test_single = fmtTest(test_data["created"].values, "weekday")
hour_test_single = fmtTest(test_data["created"].values, "hour")
test_data.loc[:, "text"] = test_data["text"].apply(rmvStops)
test_data.loc[:, "text"] = test_data["text"].apply(rmvLinks)

#create binary matrix for weekdays for test
n_test = day_test_single.shape[0]

day_test = np.zeros((n_test, 7))
for i in range(7):
    day_test[:, i] = fmtTime(day_test_single, i)

hour_test = np.zeros((n_test, 24))
for i in range(24):
    hour_test[:, i] = fmtTime(hour_test_single, i)


C:\Users\tsang\AppData\Roaming\Python\Python36\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


# Decision Tree


In [161]:
av = np.array([])
numiters = 1

for i in range(numiters):
    #Split training and validation set
    x_train, x_valid, y_train, y_valid = sklearn.model_selection.train_test_split(data["text"], data["label"],test_size = .30)
#     print (x_train)
    tknzr = TweetTokenizer(strip_handles=False, reduce_len=False)
    #Vectorize the data, create binary matrix of words
    vectorizer = CountVectorizer(binary=True,lowercase=True,tokenizer=lambda text: tknzr.tokenize(text))
#     print ()
  
    #TRAINING SET
#     words_train = words_train.reshape(words_train.shape[0], 1)
#     chars_train = chars_train.reshape(chars_train.shape[0], 1)
    
    x_train_vectors = vectorizer.fit_transform(x_train.values).toarray()
#     for i, feature in enumerate(vectorizer.get_feature_names()):
#         print(i, feature)
#     print (x_train_vectors)
#     x_train_vectors = np.append(x_train_vectors, day_train, 1)
#     x_train_vectors = np.append(x_train_vectors, hour_train, 1)
    
    #VALIDATION SET
#     words_valid = words_valid.reshape(words_valid.shape[0], 1)
#     chars_valid = chars_valid.reshape(chars_valid.shape[0], 1) 
    
    x_valid = x_valid.values
    x_valid_transformed = vectorizer.transform(x_valid).toarray()
    # validation set appended with day feature
#     x_valid_transformed = np.append(x_valid_transformed,day_valid, 1)
#     x_valid_transformed = np.append(x_valid_transformed,hour_valid, 1)
    
    #TEST SET
    x_test = vectorizer.transform(test_data["text"]).toarray()
    # test set appended with day feature
#     x_test = np.append(x_test, day_test, 1)
#     x_test = np.append(x_test, hour_test ,1)

    #Builds tree
    numTrees = 150
    dtclass = DecisionTreeClassifier(max_depth=3)
    x = dtclass.fit(x_train_vectors, y_train.values)
    
    preds_test = dtclass.predict(x_test)
    index = np.arange(0, len(preds_test))
    df = pd.DataFrame(preds_test)
    df.to_csv("plsworkdecision.csv")
    
    preds = dtclass.predict(x_valid_transformed)
    av = np.append(av, accuracy_score(preds, y_valid.values))

# print(av)
print("AVERAGE:", np.sum(av)/numiters)


AVERAGE: 0.8715596330275229


# Linear SVM

In [266]:
av = np.array([])
numiters = 1

for i in range(numiters):
    #Split training and validation set
#     print (len(day_created[0]))
    x_train, x_valid, y_train, y_valid, day_train, day_valid, hour_train, hour_valid = sklearn.model_selection.train_test_split(data["text"], data["label"], day_created, hour_created, test_size = .2)
#     use tweet tokenizer from nltk
    tknzr = TweetTokenizer(strip_handles=False, reduce_len=False)
    #Vectorize the data, create binary matrix of words
    vectorizer = CountVectorizer(binary=True,lowercase=True,tokenizer=lambda text: tknzr.tokenize(text))
    
    #TRAINING SET
#     words_train = words_train.reshape(words_train.shape[0], 1)
#     chars_train = chars_train.reshape(chars_train.shape[0], 1)
    
    x_train_vectors = vectorizer.fit_transform(x_train.values).toarray()
    x_train_vectors = np.append(x_train_vectors, day_train, 1)
    x_train_vectors = np.append(x_train_vectors, hour_train, 1)
    
    #VALIDATION SET
#     words_valid = words_valid.reshape(words_valid.shape[0], 1)
#     chars_valid = chars_valid.reshape(chars_valid.shape[0], 1) 
    
    x_valid = x_valid.values
    x_valid_transformed = vectorizer.transform(x_valid).toarray()
    # validation set appended with day feature
    x_valid_transformed = np.append(x_valid_transformed,day_valid, 1)
    x_valid_transformed = np.append(x_valid_transformed,hour_valid, 1)
#     print (x_valid_transformed)
    #TEST SET
    x_test = vectorizer.transform(test_data["text"]).toarray()
    # test set appended with day feature
    x_test = np.append(x_test, day_test, 1)
    x_test = np.append(x_test, hour_test ,1)

#     for i, feat in enumerate(vectorizer.get_feature_names()):
#         print(i, feat)

    linSVM = LinearSVC(dual=True,C=.5)
    x = linSVM.fit(x_train_vectors, y_train.values)

    preds_test = linSVM.predict(x_test)
    index = np.arange(0, len(preds_test))
    df = pd.DataFrame(preds_test)
    df.to_csv("plsworksvm.csv")
    
    preds = linSVM.predict(x_valid_transformed)
    av = np.append(av, accuracy_score(preds, y_valid.values))
#     for i in range(len(preds)):
#         if preds[i] != y_valid.values[i]:
#             print(str(y_valid.values[i]) + "; ", x_valid[i])
#             print("\n")

# print(av)
print("AVERAGE:", np.sum(av)/numiters)
# preds_test = dtclass.predict(x_test)
# print(preds_test)
#Validation accuracy

AVERAGE: 0.926605504587156


# SGDClassifier

In [50]:
av = np.array([])
numiters = 1

for i in range(numiters):
    #Split training and validation set
    x_train, x_valid, y_train, y_valid, day_train, day_valid, hour_train, hour_valid = sklearn.model_selection.train_test_split(data["text"], data["label"], day_created, hour_created, test_size = .2)

    #Vectorize the data, create tf-idf matrix of words
    vectorizer = CountVectorizer(binary=True,lowercase=True)

    #TRAINING SET
    x_train_vectors = vectorizer.fit_transform(x_train.values).toarray()
    
    x_train_vectors = np.append(x_train_vectors, day_train, 1)
    x_train_vectors = np.append(x_train_vectors, hour_train, 1)
    
    #VALIDATION SET
    x_valid = x_valid.values
    x_valid_transformed = vectorizer.transform(x_valid).toarray()
    # validation set appended with day feature
    x_valid_transformed = np.append(x_valid_transformed,day_valid, 1)
    x_valid_transformed = np.append(x_valid_transformed,hour_valid, 1)
    
    #TEST SET
    x_test = vectorizer.transform(test_data["text"]).toarray()
    # test set appended with day feature
    x_test = np.append(x_test, day_test, 1)
    x_test = np.append(x_test, hour_test ,1)

#     for i, feat in enumerate(vectorizer.get_feature_names()):
#         print(i, feat)

    sgd = SGDClassifier()
    x = sgd.fit(x_train_vectors, y_train.values)

    preds_test = sgd.predict(x_test)
    index = np.arange(0, len(preds_test))
    df = pd.DataFrame(preds_test)
    df.to_csv("plswork.csv")
    
    preds = sgd.predict(x_valid_transformed)
    av = np.append(av, accuracy_score(preds, y_valid.values))

# print(av)
print("AVERAGE:", np.sum(av)/numiters)

AVERAGE: 0.8486238532110092


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [155]:
av = np.array([])
numtimes = 15
for i in range(numtimes):
    #Split training and validation set
    X_train, X_valid, Y_train, Y_valid = sklearn.model_selection.train_test_split(data["text"], data["label"] , test_size = .2)

    #Vectorize the data, create tf-idf matrix of words
    vectorizer = CountVectorizer(binary=True)
    matrix = vectorizer.fit_transform(X_train.values)
    matrix = matrix.toarray()

    X_valid = X_valid.values
    x_valid_transformed = vectorizer.transform(X_valid).toarray()
    x_test = vectorizer.transform(test_data["text"]).toarray()

    #Builds tree

    numTrees = 100
#     dtclass = DecisionTreeClassifier(max_depth=2)
    dtclass = AdaBoostClassifier(n_estimators=numTrees, base_estimator=DecisionTreeClassifier(max_depth=2))
    x = dtclass.fit(matrix, Y_train.values)

    
    preds = dtclass.predict(x_valid_transformed)
    av = np.append(av, accuracy_score(preds, Y_valid.values))
print("AVERAGE:", np.sum(av)/numtimes)

AVERAGE: 0.8003058103975534


In [ ]:
index = np.arange(0, len(preds_test))
df = pd.DataFrame(preds_test)
df.to_csv("plswork.csv")